In [1]:
import numpy as np
import napari
import mrcfile
import tifffile
import tempfile

In [2]:
import urllib.request
def download_file_from_ftpurl(url):
    with urllib.request.urlopen(url) as response:
        with tempfile.NamedTemporaryFile(delete=False) as temp_file:
            temp_file.write(response.read())
            temp_file_path = temp_file.name

    return temp_file_path

In [3]:
#Load data
mrc_fn = download_file_from_ftpurl("https://ftp.ebi.ac.uk/empiar/world_availability/10988/data/VPP/tomograms/TS_0005.rec")

In [4]:
with mrcfile.open(mrc_fn) as f:
    data0=f.data.copy()

In [5]:
print(data0.max())
print(data0.min())

127
-128


Be careful with negative int values

Get respective cyto-ribossomes and membranes and join together 

In [6]:
with mrcfile.open("/ceph/users/ypu66991/data/Others/EMPIAR10988_Mahamid/data/VPP/labels/TS_0005_cyto_ribosomes.mrc") as f:
    labels_rib=f.data.copy().astype(np.uint8)

In [7]:
print(labels_rib.max())
print(labels_rib.min())

1
0


In [8]:
with mrcfile.open("/ceph/users/ypu66991/data/Others/EMPIAR10988_Mahamid/data/VPP/labels/TS_0005_membranes.mrc") as f:
    labels_memb=f.data.copy().astype(np.uint8)

In [9]:
print(labels_memb.max())
print(labels_memb.min())

1
0


join labels together, but be careful that there is some overlap. In overlap regions use ribos label

ribo=1
memb=2

In [10]:
labels_rib_memb = labels_rib+2*labels_memb


In [11]:
labels_rib_memb.max()

3

In [12]:
labels_rib_memb [labels_rib_memb==3] = 1 #replace overalp

ensure that voxels are not shared, otherwise we would have max2

In [13]:
labels_rib_memb.max()

2

check napari full volume

In [14]:
NV=napari.Viewer()
NV.add_image(data0)
NV.add_labels(labels_memb)
NV.add_labels(labels_rib)

<Labels layer 'labels_rib' at 0x7fbde45d8160>

Choose centre 256 cube

In [15]:
#Adjust centre
c0 = (269,345,390)

In [16]:
data0_crop = data0[ c0[0]-128: c0[0]+128,
                   c0[1]-128: c0[1]+128,
                   c0[2]-128: c0[2]+128]

In [17]:
labels_rib_memb_crop = labels_rib_memb[ c0[0]-128: c0[0]+128,
                   c0[1]-128: c0[1]+128,
                   c0[2]-128: c0[2]+128]

Check in napari

In [18]:
NV=napari.Viewer()
NV.add_image(data0_crop)
NV.add_labels(labels_rib_memb_crop)

<Labels layer 'labels_rib_memb_crop' at 0x7fbd8c0b0880>

Save

In [19]:
tifffile.imwrite("TS_0005_crop.tif", data0_crop)
tifffile.imwrite("TS_0005_ribos_membr_crop.tif", labels_rib_memb_crop)

Generate two more files for validation

In [20]:
#Adjust centre
c0 = (269,345,390+256)

In [21]:
data0_crop = data0[ c0[0]-128: c0[0]+128,
                   c0[1]-128: c0[1]+128,
                   c0[2]-128: c0[2]+128]

In [22]:
labels_rib_memb_crop = labels_rib_memb[ c0[0]-128: c0[0]+128,
                   c0[1]-128: c0[1]+128,
                   c0[2]-128: c0[2]+128]

Check in napari

In [23]:
NV=napari.Viewer()
NV.add_image(data0_crop)
NV.add_labels(labels_rib_memb_crop)

<Labels layer 'labels_rib_memb_crop' at 0x7fbd0833f850>

In [24]:
tifffile.imwrite("TS_0005_crop_val.tif", data0_crop)
tifffile.imwrite("TS_0005_ribos_membr_crop_val.tif", labels_rib_memb_crop)